# Imports

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

from pytorch_lightning import LightningModule
import torch
import torch.nn as nn
import math
from deepymod.data.base_lightning import Dataset
from torch.autograd import grad
import numpy as np
from neptune.new.types import File
import matplotlib.pyplot as plt
from datetime import datetime
from matplotlib.animation import FuncAnimation
from sklearn.linear_model import LassoCV,Lasso

from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.callbacks import LearningRateMonitor,TQDMProgressBar,EarlyStopping
from neptune.new.integrations.pytorch_lightning import NeptuneLogger


def gpu_prints():
    print("The total number of GPUs is:",torch.cuda.device_count())
    for i in range(torch.cuda.device_count()):
        print("GPU number",i,"is",torch.cuda.get_device_name(i))
        
gpu_prints()
now = datetime.now()
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")


/tmp/ipykernel_110202/3536573294.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


The total number of GPUs is: 2
GPU number 0 is NVIDIA GeForce GTX 1080 Ti
GPU number 1 is NVIDIA GeForce GTX 1080 Ti


# Regular NN

In [2]:
class SinusoidalActivation(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.pi = torch.tensor([math.pi],dtype=torch.float32,device="cuda")
    
    def forward(self,input):
        
        sinusoid = torch.sin(2*self.pi*input)
        return sinusoid
    

class NN(nn.Module):
    def __init__(self, n_in: int, n_hidden, n_out: int,init: bool) -> None:
        
        super().__init__()
        self.init = init
        self.network = self.build_network(n_in, n_hidden, n_out)
        
    def forward(self, input_: torch.Tensor):
        
        input_ = input_.requires_grad_(True)
        return self.network(input_),input_

    def init_weights(self, m):
        if isinstance(m, nn.Linear):
            torch.nn.init.xavier_uniform_(m.weight)
            torch.nn.init.zeros_(m.bias)
            

    def build_network(self, n_in, n_hidden, n_out):

        network = []
        architecture = [n_in] + n_hidden + [n_out]
        count = 0
        for layer_i, layer_j in zip(architecture, architecture[1:]):
            network.append(nn.Linear(layer_i, layer_j))
            if count ==0:
                network.append(SinusoidalActivation())
            else:
                network.append(nn.Tanh())
            count+=1
        network.pop()  # get rid of last activation function
        network = nn.Sequential(*network)
        if self.init == True:
            network.apply(self.init_weights)
        return network

In [4]:
filename        ="c=1_v=0.3c_noise=0.0_modes=0.39_0.26_t_final=10_nr=1000"

data            = np.load(f"string_data/" + filename + ".npz")
        
total_X         = torch.tensor(data["total_X"],dtype=torch.float32)
total_X         = total_X[total_X[:,0]<=4]
total_Y         = torch.tensor(data["total_Y"],dtype=torch.float32)
total_Y         = total_Y[:total_X.shape[0]]

X1              = total_X[total_X[:,0]<=2]
Y1              = total_Y[total_X[:,0]<=2]

X_validation    = total_X[total_X[:,0]>=2]
Y_validation    = total_Y[X_validation.shape[0]:,:]

X_fixed_points  = total_X[(total_X[:,1]== 0)|(total_X[:,1]== 1)]
Y_fixed_points  = total_Y[(total_X[:,1]== 0)|(total_X[:,1]== 1)]

print(Y_fixed_points)

X_MSE           = torch.cat((X1,X_fixed_points),dim=0)
Y_MSE           = torch.cat((Y1,Y_fixed_points),dim=0)

c               = torch.tensor(data["c"],dtype=torch.float32)


tensor([[ 0.0000e+00],
        [-2.7756e-17],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 2.7756e-17],
        [ 0.0000e+00],
        [-5.5511e-17],
        [ 0.0000e+00],
        [-2.7756e-17],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 1.3878e-16],
        [ 0.0000e+00],
        [ 1.6653e-16],
        [ 0.0000e+00],
        [ 1.1102e-16],
        [ 0.0000e+00],
        [ 5.5511e-17],
        [ 0.0000e+00],
        [ 5.5511e-17],
        [ 0.0000e+00],
        [ 5.5511e-17],
        [ 0.0000e+00],
        [ 5.5511e-17],
        [ 0.0000e+00],
        [ 1.1102e-16],
        [ 0.0000e+00],
        [ 1.6653e-16],
        [ 0.0000e+00],
        [-5.5511e-17],
        [ 0.0000e+00],
        [ 2.2204e-16],
        [ 0.0000e+00],
        [ 2.7756e-16],
        [ 0.0000e+00],
        [ 3.0531e-16],
        [ 0.0000e+00],
        [ 1.1102e-16],
        [ 0.0000e+00],
        [ 1.1102e-16],
        [ 0.0000e+00],
        [ 1

In [46]:
print(torch.unique(total_X[:,1]).shape)
print(X_fixed_points.shape)
#print(total_X[:50])
c = total_X[(total_X[:,1]== 0) | (total_X[:,1]== 0.1)]
d = total_Y[(total_X[:,1]== 0) | (total_X[:,1]== 0.1)| (total_X[:,1]== 0.2)]
print(total_Y[40])
print(d[2])

torch.Size([201])
torch.Size([400, 2])
tensor([0.8578])
tensor([0.8578])


In [13]:
class RegularNN(LightningModule):

    def __init__(self,filename,config):

        super().__init__()
        self.filename        = filename
        self.data            = np.load(f"string_data/" + self.filename + ".npz")
        
        self.total_X         = torch.tensor(self.data["total_X"],dtype=torch.float32)
        self.total_X         = self.total_X[self.total_X[:,0]<=4]
        self.total_Y         = torch.tensor(self.data["total_Y"],dtype=torch.float32)
        self.total_Y         = self.total_Y[:self.total_X.shape[0]]
        
        self.X_train         = self.total_X[self.total_X[:,0]<=2]
        self.X_validation    = self.total_X[self.total_X[:,0]>=2]
        
        
        self.Y_train         = self.total_Y[:self.X_train.shape[0]]
        self.Y_validation    = self.total_Y[self.X_train.shape[0]:]
        
        
        self.X_fixed_points  = self.X_validation[(self.X_validation[:,1]== 0)  |
                                                 #(self.X_validation[:,1]== 0.1)|
                                                 #(self.X_validation[:,1]== 0.2)|
                                                 #(self.X_validation[:,1]== 0.3)|
                                                 #(self.X_validation[:,1]== 0.4)|
                                                 #(self.X_validation[:,1]== 0.5)|
                                                 #(self.X_validation[:,1]== 0.6)|
                                                 #(self.X_validation[:,1]== 0.7)|
                                                 #(self.X_validation[:,1]== 0.8)|
                                                 #(self.X_validation[:,1]== 0.9)|
                                                 (self.X_validation[:,1]== 1)
                                                ]
        
        self.Y_fixed_points  = self.Y_validation[(self.X_validation[:,1]== 0) |
                                                 #(self.X_validation[:,1]== 0.1)|
                                                 #(self.X_validation[:,1]== 0.2)|
                                                 #(self.X_validation[:,1]== 0.3)|
                                                 #(self.X_validation[:,1]== 0.4)|
                                                 #(self.X_validation[:,1]== 0.5)|
                                                 #(self.X_validation[:,1]== 0.6)|
                                                 #(self.X_validation[:,1]== 0.7)|
                                                 #(self.X_validation[:,1]== 0.8)|
                                                 #(self.X_validation[:,1]== 0.9)|
                                                 (self.X_validation[:,1]== 1)
                                                 ]
        
        self.X_train = torch.cat((self.X_train,self.X_fixed_points),dim=0)
        self.Y_train = torch.cat((self.Y_train,self.Y_fixed_points),dim=0)
        
        self.c               = torch.tensor(self.data["c"],dtype=torch.float32)
        self.v               = torch.tensor(self.data["v"],dtype=torch.float32)
        self.total_x         = torch.tensor(self.data["X"],dtype=torch.float32)
        self.T               = torch.tensor(self.data["t"],dtype=torch.float32)
        self.T               = self.T[self.T<=4]
        
        self.U               = np.array(self.data["wave"])
        self.U               = self.U[:self.T.shape[0]]
        self.coefs           = self.data["coefs"]
                
        
        self.coiso              = None
        self.xi                 = None
        self.analytical_du2_dx2 = None
        self.lr                 = config["lr"]
        self.k_pde              = config["k_pde"]
        self.network            = NN(n_in=2,n_hidden=4*[50],n_out=1,init=True)

        
    def forward(self,input_):
                
        return self.network(input_)
    
    def on_train_start(self):
        
        print("Device is:",self.device)
        fig,ax1 = plt.subplots(figsize=(20,7))
        extent = np.min(self.total_x.detach().cpu().numpy()), np.max(self.total_x.detach().cpu().numpy()), np.min(self.T.detach().cpu().numpy()), np.max(self.T.detach().cpu().numpy())
        im1 = plt.imshow(self.U,origin="lower",cmap=plt.cm.magma,extent=extent,aspect="auto",interpolation=None)
        plt.colorbar()
        plt.xlabel("X")
        plt.ylabel("Time")
        plt.title(f"Real wave")
        self.logger.experiment["images/"].log(File.as_image(fig))
        plt.close()
        
       


    def training_step(self, batch,batch_idx):

        #forward pass
        
        x,target = batch
        
        prediction,coordinates = self.forward(x)
        
        second_time_deriv,theta,term1,term2 = self.compute_derivatives(prediction,coordinates)
        
        xi = self.least_squares_QR(theta,second_time_deriv)
        
        #losses
        mse_loss                            = torch.mean((prediction-target)**2) # scalar
        pde_loss                            = torch.mean((second_time_deriv+term1+term2)**2)#should be scalar as well 
        total_loss                          = mse_loss

        for i,j in enumerate(xi):
            self.log(f"Coefficient nr{i}",j,logger=True,on_epoch=True,on_step=False)
        
        self.log("MSE Loss",mse_loss,on_step=False,on_epoch=True,logger=True,prog_bar=True)
        self.log("PDE Loss",pde_loss,on_step=False,on_epoch=True,logger=True,prog_bar=True)
        self.log("Total Loss",total_loss,on_step=False,on_epoch=True,logger=True,prog_bar=True)      
        
        return total_loss

    def validation_step(self,batch,batch_idx):

        x,target = batch
        val_prediction,val_coordinates = self.forward(x)
        val_loss = torch.mean((val_prediction-target)**2)
        self.log("Validation Loss",val_loss,on_step=False,on_epoch=True,logger=True,prog_bar=True)


    def compute_derivatives(self,prediction,coords,derivs=False):
        
        du = grad(outputs=prediction, inputs=coords, grad_outputs=torch.ones_like(prediction), create_graph=True)[0]
        first_time_deriv = du[:,0:1]
        du_dx            = du[:,1:2]

        du2 = grad(outputs=first_time_deriv,inputs=coords,grad_outputs=torch.ones_like(prediction),create_graph=True)[0]
        second_time_deriv = du2[:,0:1]
        du2_dtdx          = du2[:,1:2]

        du2_dx2 = grad(outputs=du_dx,inputs=coords,grad_outputs=torch.ones_like(prediction),create_graph=True)[0][:,1:2]

        term1 = -(self.c**2-self.v**2)*du2_dx2
        term2 = +2*self.v*du2_dtdx
    
        theta = torch.reshape(torch.cat((term1,term2),dim=1),(prediction.shape[0],-1))

        if not derivs:
        
            return second_time_deriv,theta,term1,term2
        
        else:
            
            return second_time_deriv,du2_dtdx,du2_dx2

        
    def least_squares_QR(self,theta,second_deriv):
        
        Q,R = torch.linalg.qr(theta)
        xi  = torch.inverse(R) @ Q.T @ second_deriv
        return xi
    
    def configure_optimizers(self):

        optimizer = torch.optim.AdamW(self.parameters(),lr=self.lr,amsgrad=True,weight_decay=1e-6)

        return optimizer

    def train_dataloader(self):

        dataset    = Dataset(data=self.X_train, labels=self.Y_train)
        dataloader = torch.utils.data.DataLoader(dataset, batch_size=self.X_train.shape[0],drop_last=True, num_workers=0, shuffle=True)
        
        return dataloader

    def val_dataloader(self):

        val_dataset = Dataset(data=self.X_validation, labels=self.Y_validation)
        val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=self.X_validation.shape[0],
                                                     drop_last=True, num_workers=0, shuffle=False)

        return val_dataloader
    
    def analytical_derivs(self,x,time):

        if self.coiso is None:

            self.omega = np.zeros(self.coefs.shape[0])
            self.k_rev = np.zeros(self.coefs.shape[0])
            self.k_fwd = np.zeros(self.coefs.shape[0])
            self.phi   = np.zeros(self.coefs.shape[0])
            self.coiso = 1

            for i in range(self.coefs.shape[0]):

                n = i+1
                self.omega[i] = n*np.pi*(self.c**2-self.v**2)/self.c                
                self.k_rev[i] = n*np.pi*(self.c+self.v)/self.c
                self.k_fwd[i] = n*np.pi*(self.c-self.v)/self.c
                self.phi[i]   = -n*(np.pi*(self.c+self.v)/(2*self.c)-np.pi/2)



        du2_dx2  = np.zeros((len(time),len(x)))
        du2_dxdt = np.zeros((len(time),len(x)))
        du2_dt2  = np.zeros((len(time),len(x)))

        for b,t in enumerate(time):

            sum1  = 0
            sum2  = 0
            sum3  = 0

            for n in range(self.coefs.shape[0]):

                C1     = self.coefs[n] * np.sin(self.k_fwd[n]*x - self.omega[n]*t - self.phi[n])
                C2     = self.coefs[n] * np.sin(self.k_rev[n]*x + self.omega[n]*t + self.phi[n])

                sum1 += -self.k_fwd[n]**2*C1 - self.k_rev[n]**2*C2
                sum2 += +self.k_fwd[n]*self.omega[n]*C1  - self.k_rev[n]*self.omega[n]*C2
                sum3 += -self.omega[n]**2*C1 - self.omega[n]**2*C2

            du2_dx2[b,:]  = sum1
            du2_dxdt[b,:] = sum2
            du2_dt2[b,:]  = sum3

        #this analytical derivatives should be [T,X]
        return du2_dx2,du2_dxdt,du2_dt2

    def plot(self):

        
        total_output = self.forward(self.total_X.to(self.device))[0].detach().cpu().numpy().reshape(self.T.shape[0],self.total_x.shape[0])
        fig,ax1 = plt.subplots(figsize=(20,7))
        extent = np.min(self.total_x.detach().cpu().numpy()), np.max(self.total_x.detach().cpu().numpy()), np.min(self.T.detach().cpu().numpy()), np.max(self.T.detach().cpu().numpy())
        im1 = plt.imshow(total_output,origin="lower",cmap=plt.cm.magma,extent=extent,aspect="auto",interpolation=None)
        plt.colorbar()
        plt.xlabel("X")
        plt.ylabel("Time")
        plt.title(f"Full predicted wave at epoch {self.current_epoch}")
        self.logger.experiment["images/"].log(File.as_image(fig))
        plt.close()
        
        
        fig,ax1 = plt.subplots(figsize=(20,7))
        extent = np.min(self.total_x.detach().cpu().numpy()), np.max(self.total_x.detach().cpu().numpy()), np.min(self.T.detach().cpu().numpy()), np.max(self.T.detach().cpu().numpy())
        im1 = plt.imshow(np.abs(total_output-self.U),origin="lower",cmap=plt.cm.magma,extent=extent,aspect="auto",interpolation=None)
        plt.colorbar()
        plt.xlabel("X")
        plt.ylabel("Time")
        plt.title(f"Difference between true wave and predicted wave at epoch {self.current_epoch}")
        self.logger.experiment["images/"].log(File.as_image(fig))
        plt.close()
        
        
    def anim(self):
        
        def init():
            for line in lines:
                line.set_data([],[])
            return lines

        fig = plt.figure(figsize=(15,5))
        ax = plt.axes(xlim=(0,1),ylim=(-3,3))
        line, = ax.plot([], [], lw=3)
        plt.xlim(0,1)
        plt.ylim(-3,3)
        plt.xlabel("X")
        plt.ylabel("Displacement")
        plt.title(f"Prediction for epoch {self.current_epoch}",fontsize=20)
        lines = []
        lobj1 = ax.plot([],[],lw=3,label="Predicted")[0]
        lobj2 = ax.plot([],[],lw=3,label="Real")[0]
        lines.append(lobj1)
        lines.append(lobj2)
        plt.legend()
                
        anim = FuncAnimation(fig,
                    self.update_plot,
                    init_func=init,
                    frames=int(len(self.T)),
                    fargs=(lines),
                     blit=True,
                    interval=100,
                    repeat=True)
        
        plt.legend()
        anim.save(f"predictions/{self.filename}_epoch{self.current_epoch}_{now.minute}{now.second}" + ".gif",fps=30,writer="ffmpeg")
        plt.close()
        self.logger.experiment["images/"].log(File(f"predictions/{self.filename}_epoch{self.current_epoch}_{now.minute}{now.second}" + ".gif"))
        
    def on_validation_epoch_end(self):
        
#         if self.current_epoch%50 == 0:
#             with torch.enable_grad():

#                 predictions,coordinates = self.forward(self.X_train.to(self.device))

#                 #analytical derivatives--> stay constant only need to compute 1 time
#                 if self.analytical_du2_dx2 == None:

#                     self.analy_du2_dx2,self.analy_du2_dxdt,self.analy_du2_dt2 = self.analytical_derivs(self.total_x,self.T[self.T<=2])
                    
#                 #auto diff derivatives
#                 NN_du2_dt2,NN_du2_dxdt,NN_du2_dx2          = self.compute_derivatives(predictions,coordinates,derivs=True)
#                 second_time_deriv,theta,term1,term2        = self.compute_derivatives(predictions,coordinates)


#             self.xi = torch.linalg.lstsq(theta,second_time_deriv).solution        
#             for i,j in enumerate(self.xi):
#                 self.log(f"Coefficient nr{i}",j,logger=True,on_epoch=True,on_step=False)

#             NN_du2_dt2 = NN_du2_dt2.detach().cpu().numpy().reshape(int(self.T.shape[0]/2),self.total_x.shape[0])
#             NN_du2_dxdt= NN_du2_dxdt.detach().cpu().numpy().reshape(int(self.T.shape[0]/2),self.total_x.shape[0])
#             NN_du2_dx2 = NN_du2_dx2.detach().cpu().numpy().reshape(int(self.T.shape[0]/2),self.total_x.shape[0])


#             dif1 = 100 * np.median(np.abs((NN_du2_dt2[self.analy_du2_dt2>1e-5]  - self.analy_du2_dt2[self.analy_du2_dt2>1e-5])/self.analy_du2_dt2[self.analy_du2_dt2>1e-5]))        
#             dif2 = 100 * np.median(np.abs((NN_du2_dx2[self.analy_du2_dx2>1e-5]  - self.analy_du2_dx2[self.analy_du2_dx2>1e-5])/self.analy_du2_dx2[self.analy_du2_dx2>1e-5]))
#             dif3 = 100 * np.median(np.abs((NN_du2_dxdt[self.analy_du2_dxdt>1e-5] - self.analy_du2_dxdt[self.analy_du2_dxdt>1e-5])/self.analy_du2_dxdt[self.analy_du2_dxdt>1e-5]))

#             abs_dif1 = np.mean(np.abs(NN_du2_dt2 - self.analy_du2_dt2))
#             abs_dif2 = np.mean(np.abs(NN_du2_dx2 - self.analy_du2_dx2))
#             abs_dif3 = np.mean(np.abs(NN_du2_dxdt- self.analy_du2_dxdt))


#             self.log("Absolute Diff1",abs_dif1,on_step=False,on_epoch=True,prog_bar=False,logger=True)
#             self.log("Relative Diff1",dif1,on_step=False,on_epoch=True,prog_bar=False,logger=True)
#             self.log("NN_du2_dt2",np.mean(np.abs(NN_du2_dt2)),on_step=False,on_epoch=True,logger=True,prog_bar=False)    
#             self.log("Analy_du2_dt2",np.mean(np.abs(self.analy_du2_dt2)),on_step=False,on_epoch=True,logger=True,prog_bar=False)    


#             self.log("Absolute Diff2",abs_dif2,on_step=False,on_epoch=True,prog_bar=False,logger=True)
#             self.log("Relative Diff2",dif2,on_step=False,on_epoch=True,prog_bar=False,logger=True)
#             self.log("NN_du2_dx2",np.mean(np.abs(NN_du2_dx2)),on_step=False,on_epoch=True,logger=True,prog_bar=False)
#             self.log("Analy_du2_dx2",np.mean(np.abs(self.analy_du2_dx2)),on_step=False,on_epoch=True,logger=True,prog_bar=False)



#             self.log("Absolute Diff3",abs_dif3,on_step=False,on_epoch=True,prog_bar=False,logger=True)
#             self.log("Relative Diff3",dif3,on_step=False,on_epoch=True,prog_bar=False,logger=True)
#             self.log("NN_du2_dxdt",np.mean(np.abs(NN_du2_dxdt)),on_step=False,on_epoch=True,logger=True,prog_bar=False)
#             self.log("Analy_du2_dxdt",np.mean(np.abs(self.analy_du2_dxdt)),on_step=False,on_epoch=True,logger=True,prog_bar=False)


        
        if (self.current_epoch%500 == 0):
            self.plot()
            self.anim()
        else:
            pass
        
            
    def update_plot(self,i,line1,line2):
        
        a = self.forward(self.total_X.to(self.device))[0].detach().cpu().numpy()
        b   = a.reshape(self.T.shape[0],self.total_x.shape[0])
        line1.set_data(self.total_x,b[i,:])
        line2.set_data(self.total_x,self.U[i,:])
        lines = (line1,line2)
        
        return lines

In [ ]:
from pytorch_lightning.callbacks import ModelCheckpoint

neptune_logger = NeptuneLogger(api_key="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIxZjZhNTE2Ni04MDkwLTRkYjAtYjNmOS0wYTNiMDZmMWJkNDkifQ==",
                                                           project="jose.bastos/Full-Pipeline")

filename = "c=1_v=0.3c_noise=0.0_modes=0.39_0.26_t_final=10_nr=1000"

config = {
    "lr" :1e-3,
    "filename":filename,
    "k_pde":0
}

model = RegularNN(filename=filename,config=config)


print(model)

neptune_logger.experiment["model/hyper-parameters"] = config
progress_bar = TQDMProgressBar(refresh_rate=20)
lr_monitor = LearningRateMonitor(logging_interval="epoch")
checkpoint = ModelCheckpoint(dirpath="models/NN",filename=f"NN_PLUS_FIXED_POINTS")

kwargs  = {"max_epochs":-1,
            "accelerator":"gpu",
           "devices":[0],
           "auto_select_gpus":True,
            "num_sanity_val_steps":0,
            "logger" : neptune_logger,
            "enable_checkpointing": False,
            #"profiler":profiler,
            "callbacks":[lr_monitor,progress_bar]
          }


trainer = Trainer(**kwargs)
trainer.validate(model,verbose=True)
trainer.fit(model)
neptune_logger.experiment.stop()

RegularNN(
  (network): NN(
    (network): Sequential(
      (0): Linear(in_features=2, out_features=50, bias=True)
      (1): SinusoidalActivation()
      (2): Linear(in_features=50, out_features=50, bias=True)
      (3): Tanh()
      (4): Linear(in_features=50, out_features=50, bias=True)
      (5): Tanh()
      (6): Linear(in_features=50, out_features=50, bias=True)
      (7): Tanh()
      (8): Linear(in_features=50, out_features=1, bias=True)
    )
  )
)
https://app.neptune.ai/jose.bastos/Full-Pipeline/e/PIPE-223
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Validation: 0it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name    | Type | Params
---------------------------------
0 | network | NN   | 7.9 K 
---------------------------------
7.9 K     Trainable params
0         Non-trainable params
7.9 K     Total params
0.031     Total estimated model params size (MB)


────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validation Loss        0.38419485092163086
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [ ]:
trainer.save_checkpoint("models/NN/sinusoidal_wrongacti_PIPE174.ckpt")

# PINN

In [3]:
class PINN(LightningModule):

    def __init__(self,filename,config):

        super().__init__()
        self.filename        = filename
        self.data            = np.load(f"string_data/" + self.filename + ".npz")
        
        self.total_X         = torch.tensor(self.data["total_X"],dtype=torch.float32)
        self.total_X         = self.total_X[self.total_X[:,0]<=4]
        self.total_Y         = torch.tensor(self.data["total_Y"],dtype=torch.float32)
        self.total_Y         = self.total_Y[:self.total_X.shape[0]]
        
        self.X1              = self.total_X[self.total_X[:,0]<=2]
        self.Y1              = self.total_Y[self.total_X[:,0]<=2]
        
        self.X_validation    = self.total_X[self.total_X[:,0]>=2]
        self.Y_validation    = self.total_Y[self.X_validation.shape[0]:]
        
        self.X_fixed_points  = self.X_validation[(self.X_validation[:,1]== 0)  |
                                                 (self.X_validation[:,1]== 0.1)|
                                                 #(self.X_validation[:,1]== 0.2)|
                                                 #(self.X_validation[:,1]== 0.3)|
                                                 #(self.X_validation[:,1]== 0.4)|
                                                 #(self.X_validation[:,1]== 0.5)|
                                                 #(self.X_validation[:,1]== 0.6)|
                                                 #(self.X_validation[:,1]== 0.7)|
                                                 #(self.X_validation[:,1]== 0.8)|
                                                 (self.X_validation[:,1]== 0.9)|
                                                 (self.X_validation[:,1]== 1)
                                                ]
        
        self.Y_fixed_points  = self.Y_validation[(self.X_validation[:,1]== 0) |
                                                 (self.X_validation[:,1]== 0.1)|
                                                 #(self.X_validation[:,1]== 0.2)|
                                                 #(self.X_validation[:,1]== 0.3)|
                                                 #(self.X_validation[:,1]== 0.4)|
                                                 #(self.X_validation[:,1]== 0.5)|
                                                 #(self.X_validation[:,1]== 0.6)|
                                                 #(self.X_validation[:,1]== 0.7)|
                                                 #(self.X_validation[:,1]== 0.8)|
                                                 (self.X_validation[:,1]== 0.9)|
                                                 (self.X_validation[:,1]== 1)
                                                 ]
        
        self.X_MSE           = self.X_fixed_points
        self.Y_MSE           = self.Y_fixed_points
        #self.X_MSE           = torch.cat((self.X1,self.X_fixed_points),dim=0)
        #self.Y_MSE           = torch.cat((self.Y1,self.Y_fixed_points),dim=0)
        
        self.c               = torch.tensor(self.data["c"],dtype=torch.float32)
        self.v               = torch.tensor(self.data["v"],dtype=torch.float32)
        self.total_x         = torch.tensor(self.data["X"],dtype=torch.float32)
        self.T               = torch.tensor(self.data["t"],dtype=torch.float32)
        self.T               = self.T[self.T<=4]
        
        self.U               = np.array(self.data["wave"])
        self.U               = self.U[:self.T.shape[0]]
        self.coefs           = self.data["coefs"]

        
        self.coiso              = None
        self.xi                 = None
        self.analytical_du2_dx2 = None
        self.lr                 = config["lr"]
        self.k_pde              = config["k_pde"]
        self.k_mse              = config["k_mse"]
        self.network            = NN(n_in=2,n_hidden=4*[50],n_out=1,init=False)
        

        
    def forward(self,input_):
                
        return self.network(input_)
    
    def on_train_start(self):
        
        fig,ax1 = plt.subplots(figsize=(20,7))
        extent = np.min(self.total_x.detach().cpu().numpy()), np.max(self.total_x.detach().cpu().numpy()), np.min(self.T.detach().cpu().numpy()), np.max(self.T.detach().cpu().numpy())
        im1 = plt.imshow(self.U,origin="lower",cmap=plt.cm.magma,extent=extent,aspect="auto",interpolation=None)
        plt.colorbar()
        plt.xlabel("X")
        plt.ylabel("Time")
        plt.title(f"Real wave")
        self.logger.experiment["images/"].log(File.as_image(fig))
        plt.close()
        
       


    def training_step(self, batch,batch_idx):

        #forward pass
        batch1 = batch["mse"]
        batch2 = batch["pde"]
        
        x1,target1 = batch1
        x2,target2 = batch2

        prediction1,coordinates1 = self.forward(x1)
        prediction2,coordinates2 = self.forward(x2)
        
        second_time_deriv,theta,term1,term2 = self.compute_derivatives(prediction2,coordinates2)
        
        xi = self.least_squares_QR(theta,second_time_deriv)
        
        #losses
        mse_loss                            = torch.mean((prediction1-target1)**2) # scalar
        pde_loss                            = torch.mean((second_time_deriv+term1+term2)**2)#should be scalar as well 
        total_loss                          = self.k_mse*mse_loss  + self.k_pde*pde_loss

        for i,j in enumerate(xi):
            self.log(f"Coefficient nr{i}",j,logger=True,on_epoch=True,on_step=False)
        
        self.log("MSE Loss",mse_loss,on_step=False,on_epoch=True,logger=True,prog_bar=True)
        self.log("PDE Loss",pde_loss,on_step=False,on_epoch=True,logger=True,prog_bar=True)
        self.log("Total Loss",total_loss,on_step=False,on_epoch=True,logger=True,prog_bar=True)      
        
        return total_loss

    def validation_step(self,batch,batch_idx):

        x,target = batch
        val_prediction,val_coordinates = self.forward(x)
        val_loss = torch.mean((val_prediction-target)**2)
        self.log("Validation Loss",val_loss,on_step=False,on_epoch=True,logger=True,prog_bar=True)


    def compute_derivatives(self,prediction,coords,derivs=False):
        
        du = grad(outputs=prediction, inputs=coords, grad_outputs=torch.ones_like(prediction), create_graph=True)[0]
        first_time_deriv = du[:,0:1]
        du_dx            = du[:,1:2]

        du2 = grad(outputs=first_time_deriv,inputs=coords,grad_outputs=torch.ones_like(prediction),create_graph=True)[0]
        second_time_deriv = du2[:,0:1]
        du2_dtdx          = du2[:,1:2]

        du2_dx2 = grad(outputs=du_dx,inputs=coords,grad_outputs=torch.ones_like(prediction),create_graph=True)[0][:,1:2]

        term1 = -(self.c**2-self.v**2)*du2_dx2
        term2 = +2*self.v*du2_dtdx
    
        theta = torch.reshape(torch.cat((term1,term2),dim=1),(prediction.shape[0],-1))

        if not derivs:
        
            return second_time_deriv,theta,term1,term2
        
        else:
            
            return second_time_deriv,du2_dtdx,du2_dx2

        
    def least_squares_QR(self,theta,second_deriv):
        
        Q,R = torch.linalg.qr(theta)
        xi  = torch.inverse(R) @ Q.T @ second_deriv
        return xi
    
    def configure_optimizers(self):

        optimizer = torch.optim.AdamW(self.parameters(),lr=self.lr,amsgrad=True,weight_decay=1e-6)

        return optimizer

    def train_dataloader(self):

        dataset    = Dataset(data=self.X_MSE, labels=self.Y_MSE)
        dataset2   = Dataset(data=self.X_validation,labels = self.Y_validation)
        
        dataloader1 = torch.utils.data.DataLoader(dataset, batch_size=self.X_MSE.shape[0],drop_last=True, shuffle=True,num_workers=0)
        dataloader2 = torch.utils.data.DataLoader(dataset2, batch_size=self.X_validation.shape[0],drop_last=True,shuffle=True,num_workers=0)
      
        
        
        return {"mse":dataloader1,"pde":dataloader2}

    def val_dataloader(self):

        val_dataset = Dataset(data=self.X_validation, labels=self.Y_validation)
        val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=self.X_validation.shape[0],
                                                     drop_last=True, num_workers=0, shuffle=False)

        return val_dataloader
    
    def analytical_derivs(self,x,time):

        if self.coiso is None:

            self.omega = np.zeros(self.coefs.shape[0])
            self.k_rev = np.zeros(self.coefs.shape[0])
            self.k_fwd = np.zeros(self.coefs.shape[0])
            self.phi   = np.zeros(self.coefs.shape[0])
            self.coiso = 1

            for i in range(self.coefs.shape[0]):

                n = i+1
                self.omega[i] = n*np.pi*(self.c**2-self.v**2)/self.c                
                self.k_rev[i] = n*np.pi*(self.c+self.v)/self.c
                self.k_fwd[i] = n*np.pi*(self.c-self.v)/self.c
                self.phi[i]   = -n*(np.pi*(self.c+self.v)/(2*self.c)-np.pi/2)



        du2_dx2  = np.zeros((len(time),len(x)))
        du2_dxdt = np.zeros((len(time),len(x)))
        du2_dt2  = np.zeros((len(time),len(x)))

        for b,t in enumerate(time):

            sum1  = 0
            sum2  = 0
            sum3  = 0

            for n in range(self.coefs.shape[0]):

                C1     = self.coefs[n] * np.sin(self.k_fwd[n]*x - self.omega[n]*t - self.phi[n])
                C2     = self.coefs[n] * np.sin(self.k_rev[n]*x + self.omega[n]*t + self.phi[n])

                sum1 += -self.k_fwd[n]**2*C1 - self.k_rev[n]**2*C2
                sum2 += +self.k_fwd[n]*self.omega[n]*C1  - self.k_rev[n]*self.omega[n]*C2
                sum3 += -self.omega[n]**2*C1 - self.omega[n]**2*C2

            du2_dx2[b,:]  = sum1
            du2_dxdt[b,:] = sum2
            du2_dt2[b,:]  = sum3

        #this analytical derivatives should be [T,X]
        return du2_dx2,du2_dxdt,du2_dt2

    def plot(self):

        
        total_output = self.forward(self.total_X.to(self.device))[0].detach().cpu().numpy().reshape(self.T.shape[0],self.total_x.shape[0])
        fig,ax1 = plt.subplots(figsize=(20,7))
        extent = np.min(self.total_x.detach().cpu().numpy()), np.max(self.total_x.detach().cpu().numpy()), np.min(self.T.detach().cpu().numpy()), np.max(self.T.detach().cpu().numpy())
        im1 = plt.imshow(total_output,origin="lower",cmap=plt.cm.magma,extent=extent,aspect="auto",interpolation=None)
        plt.colorbar()
        plt.xlabel("X")
        plt.ylabel("Time")
        plt.title(f"Full predicted wave at epoch {self.current_epoch}")
        self.logger.experiment["images/"].log(File.as_image(fig))
        plt.close()
        
        
        fig,ax1 = plt.subplots(figsize=(20,7))
        extent = np.min(self.total_x.detach().cpu().numpy()), np.max(self.total_x.detach().cpu().numpy()), np.min(self.T.detach().cpu().numpy()), np.max(self.T.detach().cpu().numpy())
        im1 = plt.imshow(np.abs(total_output-self.U),origin="lower",cmap=plt.cm.magma,extent=extent,aspect="auto",interpolation=None)
        plt.colorbar()
        plt.xlabel("X")
        plt.ylabel("Time")
        plt.title(f"Difference between true wave and predicted wave at epoch {self.current_epoch}")
        self.logger.experiment["images/"].log(File.as_image(fig))
        plt.close()
        
        
    def anim(self):
        
        def init():
            for line in lines:
                line.set_data([],[])
            return lines

        fig = plt.figure(figsize=(15,5))
        ax = plt.axes(xlim=(0,1),ylim=(-3,3))
        line, = ax.plot([], [], lw=3)
        plt.xlim(0,1)
        plt.ylim(-3,3)
        plt.xlabel("X")
        plt.ylabel("Displacement")
        plt.title(f"Prediction for epoch {self.current_epoch}",fontsize=20)
        lines = []
        lobj1 = ax.plot([],[],lw=3,label="Predicted")[0]
        lobj2 = ax.plot([],[],lw=3,label="Real")[0]
        lines.append(lobj1)
        lines.append(lobj2)
        plt.legend()
                
        anim = FuncAnimation(fig,
                    self.update_plot,
                    init_func=init,
                    frames=int(len(self.T)),
                    fargs=(lines),
                     blit=True,
                    interval=100,
                    repeat=True)
        
        plt.legend()
        anim.save(f"predictions/{self.filename}_epoch{self.current_epoch}_{now.minute}{now.second}" + ".gif",fps=30,writer="ffmpeg")
        plt.close()
        self.logger.experiment["images/"].log(File(f"predictions/{self.filename}_epoch{self.current_epoch}_{now.minute}{now.second}" + ".gif"))
        
    def on_validation_epoch_end(self):
        
        if (self.current_epoch%500 == 0):
            self.plot()
            self.anim()
        else:
            pass
        
            
    def update_plot(self,i,line1,line2):
        
        a = self.forward(self.total_X.to(self.device))[0].detach().cpu().numpy()
        b   = a.reshape(self.T.shape[0],self.total_x.shape[0])
        line1.set_data(self.total_x,b[i,:])
        line2.set_data(self.total_x,self.U[i,:])
        lines = (line1,line2)
        
        return lines

In [6]:

filename = "c=1_v=0.3c_noise=0.0_modes=0.39_0.26_t_final=10_nr=1000"

config = {
    "lr" :1e-3,
    "filename":filename,
    "k_pde":0.01,
    "k_mse":5
}

PATH = "models/NN/NN_PLUS_FIXED_POINTS.ckpt"
model = PINN(filename=filename,config=config)
model.load_from_checkpoint(PATH,filename=filename,config=config)


print(model)






PINN(
  (network): NN(
    (network): Sequential(
      (0): Linear(in_features=2, out_features=50, bias=True)
      (1): SinusoidalActivation()
      (2): Linear(in_features=50, out_features=50, bias=True)
      (3): Tanh()
      (4): Linear(in_features=50, out_features=50, bias=True)
      (5): Tanh()
      (6): Linear(in_features=50, out_features=50, bias=True)
      (7): Tanh()
      (8): Linear(in_features=50, out_features=1, bias=True)
    )
  )
)


In [ ]:
neptune_logger = NeptuneLogger(api_key="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIxZjZhNTE2Ni04MDkwLTRkYjAtYjNmOS0wYTNiMDZmMWJkNDkifQ==",
                                                                                          project="jose.bastos/Full-Pipeline")

neptune_logger.experiment["model/hyper-parameters"] = config
progress_bar = TQDMProgressBar(refresh_rate=20)
lr_monitor = LearningRateMonitor(logging_interval="epoch")
                               

    
kwargs  = {"max_epochs":-1,
            "accelerator":"gpu",
           "devices":[0],
           "auto_select_gpus":True,
            "num_sanity_val_steps":0,
            "logger" : neptune_logger,
            "enable_checkpointing": False,
            "enable_progress_bar":False,
            #"profiler":profiler,
            "callbacks":[lr_monitor]
          }





trainer = Trainer(**kwargs)
trainer.validate(model,ckpt_path=PATH,verbose=True)
trainer.fit(model,ckpt_path=PATH)
neptune_logger.experiment.stop()


https://app.neptune.ai/jose.bastos/Full-Pipeline/e/PIPE-228
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Restoring states from the checkpoint path at models/NN/NN_PLUS_FIXED_POINTS.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
Loaded model weights from checkpoint at models/NN/NN_PLUS_FIXED_POINTS.ckpt
/home/jose/anaconda3/envs/DeepMod/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
Restoring states from the checkpoint path at models/NN/NN_PLUS_FIXED_POINTS.ckpt
/home/jose/anaconda3/envs/DeepMod/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1721: UserWarning: Be aware that when using `ckpt_path`, callb

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validation Loss      0.00011769861885113642
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


/home/jose/anaconda3/envs/DeepMod/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/jose/anaconda3/envs/DeepMod/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1938: PossibleUserWarning: The number of training samples (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(
/home/jose/anaconda3/envs/DeepMod/lib/python3.9/site-packages/pytorch_lightning/utilities/data.py:72: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 800. To avoid any miscalculations, use `self.log

In [ ]:
torch.save("models/PINN/Boundary_only_PIPE220.ckpt")